「PyTorch入門  8. クイックスタート」
===============================================================
【原題】Learn the Basics

【原著】
[Suraj Subramanian](https://github.com/suraj813)、[Seth Juarez](https://github.com/sethjuarez/) 、[Cassie Breviu](https://github.com/cassieview/) 、[Dmitry Soshnikov](https://soshnikov.com/)、[Ari Bornstein](https://github.com/aribornstein/) 


【元URL】https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

【翻訳】電通国際情報サービスISID AIトランスフォーメーションセンター　小川 雄太郎

【日付】2021年03月20日

【チュトーリアル概要】

本チュートリアルでは、機械学習を実行するためのPyTorchの各APIを紹介します。詳細を知りたい方は、各セクションのリンク先をご参照ください。

---


【注意】

ディープラーニングフレームワークの経験があり、ディープラーニングの実装に慣れている方は、本チュートリアルをまず確認し、PyTorchのAPIに慣れてください。

<br>

ディープラーニングフレームワークを利用した実装に初めて取り組む方は、本チュートリアルからではなく、

[「PyTorch入門  1. テンソル」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/1_1_tensor_tutorial_jp.ipynb)から、1 stepずつ進めてください。



---


データの取り扱い
-----------------

PyTorchではデータを取り扱う際に、基本的な要素が2つ存在します。

``torch.utils.data.Dataset``と``torch.utils.data.DataLoader``です。

<br>

``Dataset`` は各ラベルと、それに対応するサンプルデータを保持します。

``DataLoader`` は``Dataset``をイテレーティブに（＝反復処理可能に）操作できるようラップしたものになります。

<br>

（詳細）
https://pytorch.org/docs/stable/data.html





In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorchには以下に示すようなドメイン固有のライブラリが存在し、それぞれにデータセットが用意されています。


本チュートリアルでは、TorchVisionのデータセットを使用します。

- [TorchText](https://pytorch.org/text/stable/index.html)
- [TorchVision](https://pytorch.org/vision/stable/index.html)
- [TorchAudio](https://pytorch.org/audio/stable/index.html)


``torchvision.datasets`` モジュールには、画像データの ``Dataset`` オブジェクトがたくさん用意されています。

例えば、CIFAR, COCOなどです ([データセット一覧はこちらから](https://pytorch.org/docs/stable/torchvision/datasets.html))。

本チュートリアルでは、FashionMNISTデータセットを使用します。

TorchVisionの すべての``Dataset`` には2つの引数があります。

``transform`` と ``target_transform`` であり、それぞれサンプルとラベルの変換処理を行います。

In [3]:
# 訓練データをdatasetsからダウンロード
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# テストデータをdatasetsからダウンロード
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


DataLoaderの引数としてDatasetを渡します。


DataLoaderはDatasetをイテレート処理できるようにラップしたものです。

バッチ処理、サンプリング、シャッフル、マルチプロセスでのデータ読み込み等をサポートしています。




以下では、バッチサイズを64と定義します。

つまり、データローダの反復要素は、64個の特徴量とラベルから構成されるバッチを返すことになります。

In [4]:
batch_size = 64

# データローダーの作成
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


さらなる詳細は、[「PyTorch入門  2. データセットとデータローダー」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_2_data_tutorial_jp.ipynb
)をご覧ください。


--------------




モデルの構築
------------------

PyTorchでニューラルネットワークの形を定義する際には、[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承します。


``__init__`` 関数で、ネットワークの各レイヤーを定義し、データの順伝搬を``forward`` 関数に定義します。

なお処理を高速化するために、可能であればニューラルネットワークをGPU上へ移動させます。



In [5]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)



さらなる詳細は、[「PyTorch入門 4. モデル構築」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_4_buildmodel_tutorial_js.ipynb
)をご覧ください。



--------------




モデルパラメータの最適化
----------------------------------------

ニューラルネットワークモデルを訓練するためには、

損失関数：[loss function](<https://pytorch.org/docs/stable/nn.html#loss-functions)と、最適化手法：[optimizer](https://pytorch.org/docs/stable/optim.html)が必要です。



In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

1回の訓練ループにおいてモデルは、まず訓練データのバッチに対して推論を実行して損失誤差を計算し、

その後、損失誤差をバックプロパゲーションして、モデルパラメータを更新します。




In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

また、モデルがうまく学習していることを確認するために、テストデータセットに対するモデルの性能も確認します。



In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

訓練プロセスでは複数イテレーションを実行します。

各エポックの間、モデルはより良い予測ができるようにパラメータを学習します。

エポックごとにモデルの正解率と損失を出力して、正解率が向上し、損失が低下していっているかを確認します。




In [9]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301979  [    0/60000]
loss: 2.295124  [ 6400/60000]
loss: 2.284376  [12800/60000]
loss: 2.288383  [19200/60000]
loss: 2.268149  [25600/60000]
loss: 2.254719  [32000/60000]
loss: 2.270254  [38400/60000]
loss: 2.246339  [44800/60000]
loss: 2.241662  [51200/60000]
loss: 2.233782  [57600/60000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 0.034994 

Epoch 2
-------------------------------
loss: 2.231522  [    0/60000]
loss: 2.210618  [ 6400/60000]
loss: 2.192281  [12800/60000]
loss: 2.215005  [19200/60000]
loss: 2.170340  [25600/60000]
loss: 2.148389  [32000/60000]
loss: 2.188285  [38400/60000]
loss: 2.142316  [44800/60000]
loss: 2.142076  [51200/60000]
loss: 2.123532  [57600/60000]
Test Error: 
 Accuracy: 49.5%, Avg loss: 0.033262 

Epoch 3
-------------------------------
loss: 2.127300  [    0/60000]
loss: 2.077713  [ 6400/60000]
loss: 2.049166  [12800/60000]
loss: 2.102314  [19200/60000]
loss: 2.012621  [25600/60000]
loss: 1.986534  [32000/600

さらなる詳細は、[「PyTorch入門 6. 最適化」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_6_optimization_tutorial_js.ipynb)をご覧ください。


--------------




モデルの保存
-------------

モデルを保存する一般的な方法は、モデルの内部状態の辞書（モデルのパラメータを含む）をシリアル化する手法です。


In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


---


モデルの読み込み
----------------------------

モデルの読み込む際には、まずモデルの構造を再作成し、そのインスタンスに、保存しておいた状態辞書をロードします。


In [11]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

これでモデルは推論可能な状態です。




In [12]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


さらなる詳細は、[「PyTorch入門 7. モデルの保存・読み込み」](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/0_Learn%20the%20Basics/0_7_saveloadrun_tutorial_js.ipynb)をご覧ください。





以上。